# Conditional Kaplan Meier Estimation

Before running the notebook make sure that:

1. Created a python/conda environment according to the specifications
2. Set your working directory (`project_dir`) to the directory where you unpacked the .zip file
3. You might want to add a shebang line to the ./utils (however using it from notebooks with a specified environment should work as well)

The notebook genereates data, trains models and saves them into the ./data directory. Note that at times you will need to overwrite existing files (this is to ensure that you do not accidentally overwrite files that took a long time training - aka. model weights)

See R-Scrips for visualisations (by default, all visualisations should already be in the zip folder)

In [1]:
project_dir = ''

In [2]:
# Housekeeping
import os
os.chdir(project_dir)

# Wrangling
import numpy as np
import pandas as pd
from itertools import product as itp
import pickle

# Stats
from lifelines import KaplanMeierFitter, CoxPHFitter
from sklearn.preprocessing import StandardScaler

# TF & Keras
import tensorflow as tf
from tensorflow import keras

In [3]:
%load_ext autoreload
%autoreload 2

from main.utils.conditional_km import DeepKaplanMeier
from main.utils.metrics import calculate_concordance_surv
train_ = False

In [4]:
df = pd.read_csv('data/application/application_training_smart.csv')
y_target = df.tevent.astype(int)
y_censoring = np.where(df.event == 1, False, True)

df_val = pd.read_csv('data/application/application_test_smart.csv')
y_target_val = df_val.tevent.astype(int)
y_censoring_val = np.where(df_val.event == 1, False, True)

In [5]:
features_unscaled = df.loc[:, df.columns[3:]]
features_unscaled_val = df_val.loc[:, df_val.columns[3:]]

features_numeric = features_unscaled.max(axis=0) > 1

numeric_ = features_unscaled.loc[:,features_numeric.values]
binary_ = features_unscaled.loc[:,~features_numeric.values].to_numpy()

numeric_val = features_unscaled_val.loc[:,features_numeric.values]
binary_val = features_unscaled_val.loc[:,~features_numeric.values].to_numpy()

scaler = StandardScaler()
scaler.fit(numeric_)

numeric_trans = scaler.transform(numeric_)
numeric_trans_val = scaler.transform(numeric_val)

features = np.concatenate([numeric_trans, binary_], axis=1)
features_val = np.concatenate([numeric_trans_val, binary_val], axis=1)

In [6]:
total_periods = df.tevent.max()
input_dim = (features.shape[1], )
input_shape = input_dim

In [7]:
init_mod = DeepKaplanMeier(int(total_periods))

In [8]:
y_matrix, weight_matrix = init_mod.prepare_survival(list(y_target), list(y_censoring))
y_matrix_val, weight_matrix_val = init_mod.prepare_survival(list(y_target_val), list(y_censoring_val))

train_dataset = tf.data.Dataset.from_tensor_slices(tuple([features] + [y_matrix.reshape(-1,36)] + [weight_matrix.reshape(-1,36)] + [y_target]))
val_dataset = tf.data.Dataset.from_tensor_slices(tuple([features_val] + [y_matrix_val.reshape(-1,36)] + [weight_matrix_val.reshape(-1,36)] + [y_target_val]))

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-10-20 18:01:53.039306: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-20 18:01:53.039407: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
trial_dict = {}

In [10]:
if train_:
    input_shape = input_dim

    total_trials = 150

    # Optimize layer 1-3 plus additional layer
    # Optimize learning rate
    # Optimize batchsize
    layer_sizes = np.arange(3,13, 3)
    learning_rate = [5e-02, 1e-02, 5e-03, 1e-03]
    batch_sizes = [32,64,128,256]

    np.random.seed(123)
    for trial_ in range(total_trials):
        trial_dict[f"trial_{trial_}"] = {}

        check_indices = np.ceil(np.random.uniform(size=6)*4)-1
        check_indices = check_indices.astype(int)

        hp_unit_1 = layer_sizes[check_indices[0]]
        hp_unit_2 = layer_sizes[check_indices[1]]
        hp_unit_3 = layer_sizes[check_indices[2]]
        hp_unit_4 = layer_sizes[check_indices[3]]

        hp_learning_rate = learning_rate[check_indices[4]]
        hp_batch = batch_sizes[check_indices[5]]

        trial_dict[f"trial_{trial_}"]['hp_unit_1'] = hp_unit_1
        trial_dict[f"trial_{trial_}"]['hp_unit_2'] = hp_unit_2
        trial_dict[f"trial_{trial_}"]['hp_unit_3'] = hp_unit_3
        trial_dict[f"trial_{trial_}"]['hp_unit_4'] = hp_unit_4
        trial_dict[f"trial_{trial_}"]['hp_learning_rate'] = hp_learning_rate
        trial_dict[f"trial_{trial_}"]['hp_batch'] = hp_batch
        trial_dict[f"trial_{trial_}"]['epochs'] = {}

        print(trial_dict[f"trial_{trial_}"])

        model_raw = init_mod.return_comp_graph_raw(input_shape=input_shape, 
                                                hidden_units=[hp_unit_1, hp_unit_2, hp_unit_3],
                                                preoutput=hp_unit_4)

        upper_bound = np.random.uniform(1,3.5)
        trial_dict[f"trial_{trial_}"]['weight'] = upper_bound
        weight_decrease = 1 + np.linspace(0,upper_bound,36)
        weight_array = weight_decrease
        best_epoch_loss = 0

        #### Update variables ####
        
        batch_size=hp_batch
        optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate)
        loss_objective = keras.losses.BinaryCrossentropy()
        epoch_loss_metric = keras.metrics.Mean()

        train_ds = train_dataset.batch(batch_size)
        check_dataset = train_dataset.shuffle(10)
        check_dataset = check_dataset.take(775)
        check_ds = check_dataset.batch(775)

        for epoch in range(75):
            print(f"Epoch: {epoch}")
            if best_epoch_loss < 0.5 and epoch > 10:
                continue
            
            for step, (features_, _labels, _weights, _) in enumerate(train_ds):
                with tf.GradientTape() as tape:
                        preds_ = model_raw(features_, training=True)
                        losses = [loss_objective(_labels[:,i], preds_[i], sample_weight=tf.reshape(_weights[:,i], (-1,1))) for i in range(len(preds_))]
                        losses = [tf.multiply(loss_, weight_) for loss_, weight_ in zip(losses, weight_array)]
                        gradients = tape.gradient(losses, model_raw.trainable_variables)
                        optimizer.apply_gradients(zip(gradients, model_raw.trainable_variables))

            for features_, _labels, _weights, true_ in check_ds:
                preds_test = model_raw(features_)
                preds_test = pd.DataFrame(list(map(np.ravel, preds_test)))

                nom_, denom_, val_ = calculate_concordance_surv(y_target_val, 
                                                                y_censoring_val, 
                                                                preds_test)

                
                epoch_loss_metric = val_
                
            trial_dict[f"trial_{trial_}"]['epochs'][f'epoch_{epoch}'] = epoch_loss_metric
            best_epoch_loss = max(best_epoch_loss, epoch_loss_metric)
            print(f'best validation result is: {best_epoch_loss}')
            epoch_loss_metric = 0


        trial_dict[f"trial_{trial_}"]['best_validation_result'] = best_epoch_loss
        with open('./model_tuning/manual_gridsearch/tuning_smart_data.pkl', 'wb') as con:
            pickle.dump(trial_dict, con)

In [11]:
if train_:
        
    hp_unit_1  = 4
    hp_unit_2  = 12
    hp_unit_3  = 12
    hp_unit_4  = 12
    hp_batch =  256
    hp_learning_rate = 0.005
    hp_weight = 0
    hp_epochs = 30

    model_raw = init_mod.return_comp_graph_raw(input_shape=input_shape, 
                                            hidden_units=[hp_unit_1, hp_unit_2, hp_unit_3],
                                            preoutput=hp_unit_4)

    batch_size=hp_batch
    optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate)
    loss_objective = keras.losses.BinaryCrossentropy()
    train_ds = train_dataset.batch(batch_size)

    weight_decrease = 1 + np.linspace(0,hp_weight,36)
    weight_array = weight_decrease

    for epoch in range(hp_epochs):
        print(f"Epoch: {epoch}")
        for step, (features_, _labels, _weights, _) in enumerate(train_ds):
            with tf.GradientTape() as tape:
                    preds_ = model_raw(features_, training=True)
                    losses = [loss_objective(_labels[:,i], preds_[i], sample_weight=tf.reshape(_weights[:,i], (-1,1))) for i in range(len(preds_))]
                    losses = [tf.multiply(loss_, weight_) for loss_, weight_ in zip(losses, weight_array)]
                    gradients = tape.gradient(losses, model_raw.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model_raw.trainable_variables))

    model_smart = model_raw
else:
    model_smart = tf.keras.models.load_model('data/model_weights/model_smart_application.keras')

In [12]:
preds_test = model_smart(features_val)
preds_test = pd.DataFrame(list(map(np.ravel, preds_test)))

nom_, denom_, val_ = calculate_concordance_surv(y_target_val, 
                                        y_censoring_val, 
                                        preds_test)
print(f"C-index is: {np.round(val_, 3)}")

C-index is: 0.693


In [13]:
preds_test.to_csv('data/application/results_smart_ckm.csv', index=False)

## Cox PH Model

In [14]:
df_cox = df.drop(columns=['rowid'])
df_cox_test = df_val.drop(columns=['event'])

In [15]:
cph = CoxPHFitter()
cph.fit(df_cox, duration_col='tevent', event_col='event')

preds_cox = cph.predict_survival_function(df_cox_test)

In [16]:
preds_cox.to_csv('data/application/results_smart_cox.csv', 
index=False)

In [17]:
preds_cox = cph.predict_survival_function(df_cox_test)

nom_, denom_, val_ = calculate_concordance_surv(y_target_val, 
                                                y_censoring_val, 
                                                preds_cox)
print(f"Concordance is: {np.round(val_,3)}")

Concordance is: 0.669


In [18]:
kaplan_meier = KaplanMeierFitter()
kaplan_meier.fit(y_target_val,
                 ~y_censoring_val)

kaplan_meier_survival = kaplan_meier.survival_function_

# Export for plotting
(kaplan_meier_survival.reset_index()
                      .to_csv('data/application/results_smart_km.csv', 
                      index=False))